In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from scipy import stats
import eda

## 유동인구

**성연령**

In [2]:
age = eda.flow_concat('AGE','../data/01_SKT/*.csv')
age = eda.flow_preprocessing(age)
age = eda.flow_melting(age)
age['AGE_CD'] = age['VAR'].apply(lambda x: x[-3:] if x[-4] == '_' else x[-4:])
age['SEX_CD'] = age['VAR'].str[0].map({'M':'M', 'W':'F'})
age = age[['STD_YMD', 'GU_NM', 'HDONG_NM', 'SEX_CD', 'AGE_CD', 'POP_CNT']]
age_dict = {x: int(x[:2])for x in sorted(age['AGE_CD'].unique())}
age['AGE_CD'] = age['AGE_CD'].map(age_dict)
age['AGE_CD'] = age['AGE_CD'].apply(lambda x: 20 if x < 20 else x)
age['AGE_CD'] = age['AGE_CD'].apply(lambda x: 65 if x > 65 else x)
age = age.groupby(['STD_YMD', 'GU_NM', 'HDONG_NM'])['POP_CNT'].sum().reset_index()
age_per = pd.read_csv('../data/08_Percent_Matrix/age_per.csv')
age = pd.concat([age, age_per],axis=1)


print(age.shape)
age.head()

(16629, 14)


,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,AG_30_M,AG_40_M,AG_50_M,AG_60_M
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,0.085641,0.131000,0.129117,0.101335
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,0.106579,0.147573,0.143406,0.105276
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,0.089873,0.129602,0.128439,0.105352
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,0.106474,0.141090,0.126571,0.103969
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,0.100490,0.136893,0.139035,0.116909


**시간**

In [3]:
time = eda.flow_concat('TIME','../data/01_SKT/*.csv')
time = eda.flow_preprocessing(time)

print(time.shape)
time.head()

(16629, 27)


,STD_YMD,GU_NM,HDONG_NM,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,2019-02-01,서울 중구,소공동,1496.35,1022.83,706.51,547.77,1673.02,3561.05,9195.82,...,12355.91,13096.69,9317.64,15029.74,14135.83,9074.02,6032.26,6047.05,4806.65,2956.11
1,2019-02-01,서울 중구,회현동,1692.37,1117.26,839.57,653.41,1920.52,4687.95,10936.47,...,18210.30,20292.65,12527.29,21454.63,18214.50,11914.26,9800.64,9341.24,6884.91,4034.56
2,2019-02-01,서울 중구,명동,2226.91,1726.10,1145.43,894.97,2513.32,5268.51,13857.93,...,20332.71,21458.64,14434.56,25855.14,23107.23,13460.05,9797.40,8719.30,7203.45,4845.48
3,2019-02-01,서울 중구,필동,698.73,440.70,385.99,275.64,497.41,1990.42,4679.79,...,6900.66,9789.54,6205.74,13277.25,12101.51,5698.58,4464.46,4088.53,3385.14,1746.28
4,2019-02-01,서울 중구,장충동,768.34,447.23,429.90,349.46,507.74,1310.67,2956.49,...,5187.02,5211.60,3656.54,6841.98,5810.71,4463.99,2807.25,2626.12,2415.41,1398.94


In [4]:
total = time.loc[: , 'TMST_00':].sum(axis=1)
for i in time.columns[3:].str[-2:]:
    time[f'TMST_{i}'] = time[f'TMST_{i}'] / total

In [5]:
flow = pd.merge(age, time, on=['STD_YMD', 'GU_NM', 'HDONG_NM'])

<b> =============================

연령+성별 데이터

1. 20년 데이터 slicing

In [6]:
df_flow_age = flow.query('STD_YMD >= "2020-02-01"')[flow_age]

NameError: name 'flow_age' is not defined

In [ ]:
df_flow_age.insert(0, 'STD_YMD', flow['STD_YMD'])

In [ ]:
df_flow_age.insert(1, 'GU_NM', flow['GU_NM'])

In [ ]:
df_flow_age.insert(2, 'HDONG_NM', flow['HDONG_NM'])

In [ ]:
df_flow_age.head()

전년대비 감소율 = (20년 일별-19년 동월 평균)/19년 동월 평균

In [ ]:
df_flow_age.groupby(['STD_YMD'])['AG_20_F'].mean().reset_index() - flow.query('STD_YMD <= "2019-05-31"').groupby(['STD_YMD'])['AG_20_F'].mean().reset_index()

In [ ]:
flow.query('STD_YMD <= "2019-05-31"').groupby(['STD_YMD'])['AG_20_F'].mean().reset_index()

## 카드매출

**내국인**

In [7]:
resid = pd.read_csv('../data/02_Shinhan_Card/shinhan_dataset(내국인).txt', sep='\t')
resid = eda.card_preprocessing(resid)
resid.rename(columns={'STD_DD':'STD_YMD', 'DONG_NM':'HDONG_NM'}, inplace=True)
sales = resid.groupby(['STD_YMD', 'GU_NM', 'HDONG_NM'])['USE_AMT'].sum().reset_index()

renaming = ['LODGE', 'LEIS_ITEM', 'LEIS', 'ELEC', 'KITCH', 'FUEL', 'OPTICAL', 'DIST',
            'OFFICE', 'CAR_SERVICE', 'MEDIAN', 'HYGIENE', 'REST', 'GROCERY', 'REPAIR',
            'HOBBY', 'FURN', 'APPL', 'CLOTHES', 'ACC', 'BOOK', 'CROP', 'CAR_SALES']
df_list = []
for name in list(map(lambda x: x.lower(), renaming)):
    df_list.append(pd.read_csv(f'../data/08_Percent_Matrix/{name}_per.csv'))

up_per = pd.concat(df_list, axis=1)
up_sales = eda.upjong_sales(resid)
sales = pd.concat([sales, up_sales, up_per], axis=1)

print(sales.shape)
sales.head()

(16629, 257)


,STD_YMD,GU_NM,HDONG_NM,USE_AMT,LODGE_AMT,LEIS_ITEM_AMT,LEIS_AMT,ELEC_AMT,KITCH_AMT,FUEL_AMT,...,SH_CAR_SALES_20_F,SH_CAR_SALES_30_F,SH_CAR_SALES_40_F,SH_CAR_SALES_50_F,SH_CAR_SALES_60_F,SH_CAR_SALES_20_M,SH_CAR_SALES_30_M,SH_CAR_SALES_40_M,SH_CAR_SALES_50_M,SH_CAR_SALES_60_M
0,2019-02-01,대구 수성구,고산1동,411993,0,3443,22485,0,3210,13734,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,2019-02-01,대구 수성구,고산2동,386930,0,6772,3677,0,0,122264,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,2019-02-01,대구 수성구,고산3동,336754,0,10945,5313,0,0,66983,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,2019-02-01,대구 수성구,두산동,681681,7241,2453,16490,0,1542,37363,...,0.178383,0.0,0.0,0.0,0.007519,0.0,0.0,0.469173,0.0,0.344925
4,2019-02-01,대구 수성구,만촌1동,540787,17450,797,9833,243,2844,69767,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [8]:
table = pd.merge(flow, sales, on=['STD_YMD', 'GU_NM', 'HDONG_NM'])
table.head()

,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,...,SH_CAR_SALES_20_F,SH_CAR_SALES_30_F,SH_CAR_SALES_40_F,SH_CAR_SALES_50_F,SH_CAR_SALES_60_F,SH_CAR_SALES_20_M,SH_CAR_SALES_30_M,SH_CAR_SALES_40_M,SH_CAR_SALES_50_M,SH_CAR_SALES_60_M
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,...,0.178383,0.0,0.0,0.0,0.007519,0.0,0.0,0.469173,0.0,0.344925
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


## 유통

In [9]:
logistics = pd.read_excel('../data/04_GS_Retail/gs_dataset.xlsx',
                          sheet_name='TB_DD_STR_PURCH_CLS',
                          parse_dates=['OPER_DT'])
logistics['BOR'] = logistics['PVN'].str[:2] + ' ' + logistics['BOR']
logistics.rename(columns={'OPER_DT':'STD_YMD', 'BOR':'GU_NM', 'ADMD':'HDONG_NM'},
                 inplace=True)
logistics.drop('PVN', axis=1, inplace=True)
print(logistics.shape)
logistics.head()

(14460, 12)


,STD_YMD,GU_NM,HDONG_NM,AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P
0,2019-02-01,대구 수성구,고산1동,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,2019-02-01,대구 수성구,고산2동,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,2019-02-01,대구 수성구,고산3동,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,2019-02-01,대구 수성구,두산동,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,2019-02-01,대구 수성구,만촌1동,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719


In [10]:
logistics.columns = ['STD_YMD', 'GU_NM', 'HDONG_NM', 'AMT_IND', 'GS_MEAL',
                     'GS_SNACK', 'GS_DRINK', 'GS_HOME', 'GS_HEALTH', 'GS_HOBBY', 
                     'GS_SOCIAL', 'GS_PREG']

In [11]:
table = pd.merge(table, logistics, on=['STD_YMD', 'GU_NM', 'HDONG_NM'], how='left') \
          .fillna(0)

In [12]:
table.head()

,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,...,SH_CAR_SALES_60_M,AMT_IND,GS_MEAL,GS_SNACK,GS_DRINK,GS_HOME,GS_HEALTH,GS_HOBBY,GS_SOCIAL,GS_PREG
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,...,0.000000,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,...,0.000000,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,...,0.000000,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,...,0.344925,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,...,0.000000,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719


## 물류

In [13]:
delivery = pd.read_excel('../data/05_CJ_Olive_Networks/cj_dataset.xlsx')
delivery = eda.delivery_preprocessing(delivery)
delivery_group = delivery.groupby(['STD_YMD', 'GU_NM', 'HDONG_NM'])['INV_CNT'].sum().reset_index()
sharing_index = pd.read_csv('../data/sharing_index.csv', parse_dates=['STD_YMD'])
delivery_group = pd.merge(sharing_index, delivery_group, how='left').fillna(0)
delivery_group['INV_CNT'] = delivery_group['INV_CNT'].astype('int')
delivery_per = pd.read_csv('../data/08_Percent_Matrix/delivery_per.csv')
delivery = pd.concat([delivery_group, delivery_per], axis=1)

print(delivery.shape)
delivery.head()

(16629, 14)


,STD_YMD,GU_NM,HDONG_NM,INV_CNT,CJ_FURN,CJ_BOOK,CJ_APPL,CJ_HEALTH,CJ_SPORT,CJ_FOOD,CJ_PREG,CJ_CLOTH,CJ_ACC,CJ_BEAUTY
0,2019-02-01,대구 수성구,고산1동,333,0.006006,0.342342,0.042042,0.093093,0.012012,0.108108,0.003003,0.195195,0.045045,0.153153
1,2019-02-01,대구 수성구,고산2동,220,0.018182,0.331818,0.081818,0.081818,0.004545,0.059091,0.000000,0.168182,0.100000,0.154545
2,2019-02-01,대구 수성구,고산3동,348,0.040230,0.264368,0.060345,0.137931,0.008621,0.089080,0.005747,0.178161,0.074713,0.140805
3,2019-02-01,대구 수성구,두산동,137,0.007299,0.328467,0.051095,0.116788,0.000000,0.043796,0.000000,0.211679,0.102190,0.138686
4,2019-02-01,대구 수성구,만촌1동,186,0.010753,0.301075,0.064516,0.112903,0.000000,0.075269,0.005376,0.252688,0.064516,0.112903


In [14]:
table = pd.merge(table, delivery, on=['STD_YMD', 'GU_NM', 'HDONG_NM'], how='left')

## 질병

**서울**

In [80]:
seoul = pd.read_csv('../data/06_Epidemic_Datasets/Seoul_Covid.csv')
seoul = eda.seoul_preprocessing(seoul)
seoul['S_PT_CNT'] = 1
seoul = seoul[['S_PT_CNT']]
seoul = seoul.groupby('STD_YMD')['S_PT_CNT'].sum().reset_index()
period = pd.date_range('2020-01-24', '2020-05-31', freq='d')
period = pd.DataFrame(period, columns=['STD_YMD'])
seoul = pd.merge(period, seoul, how='left').fillna(0)
seoul['S_PT_CNT'] = seoul['S_PT_CNT'].astype('int')
seoul['CUM_CNT'] = seoul['S_PT_CNT'].cumsum()
seoul = seoul.query('STD_YMD >= "2020-02-01"')
seoul = seoul[['STD_YMD', 'S_PT_CNT']]

print(seoul.shape)
seoul.head()

TypeError: cannot do positional indexing on DatetimeIndex with these indexers [2020-05-31] of type str

In [ ]:
daegu = pd.read_csv('../data/06_Epidemic_Datasets/Daegu_Covid.csv', 
                    encoding='cp949', parse_dates=['날짜'])
daegu.rename(columns={'날짜':'STD_YMD', '당일 확진자 수':'D_PT_CNT'}, inplace=True)
daegu = daegu[['STD_YMD', 'D_PT_CNT']]
daegu = pd.merge(period, daegu, how='left').fillna(0)
daegu['D_PT_CNT'] = daegu['D_PT_CNT'].astype('int')
daegu['CUM_CNT'] = daegu['D_PT_CNT'].cumsum()
daegu = daegu.query('STD_YMD >= "2020-02-01"')
daegu = daegu[['STD_YMD', 'D_PT_CNT']]

print(daegu.shape)
daegu.head()

In [ ]:
table = pd.merge(table, seoul, how='left').merge(daegu, how='left').fillna(0)
table['S_PT_CNT'] = table['S_PT_CNT'].astype('int')
table['D_PT_CNT'] = table['D_PT_CNT'].astype('int')

## 날씨

In [ ]:
df_list = [pd.read_csv('../data/07_External_Datasets/weather_{}.csv' \
                       .format(i), encoding='cp949') for i in [2019, 2020]]
weather = pd.concat(df_list)
weather = weather.fillna(0)
weather['RAIN'] = weather['일강수량(mm)'].apply(lambda x: 1 if x > 0 else 0)
weather = weather[['지점명', '일시', '평균기온(°C)', 'RAIN']]
weather.rename(columns={'지점명':'CT_NM', '일시':'STD_YMD',
                        '평균기온(°C)':'AV_TEMP'}, inplace=True)
weather['STD_YMD'] = weather['STD_YMD'].astype('datetime64')
print(weather.shape)
weather.head()

In [ ]:
table = pd.merge(table.assign(CT_NM=table['GU_NM'].str[:2]), weather, on=['STD_YMD', 'CT_NM'])

In [ ]:
table = pd.concat([table.drop(['POP_CNT', 'USE_AMT', 'AMT_IND', 'INV_CNT', 'CT_NM'], axis=1),
                   table[['POP_CNT', 'AMT_IND', 'INV_CNT', 'USE_AMT']]], axis=1)

**Sinhyuk part**

**GS**

In [19]:
table  = table.set_index('STD_YMD')

In [20]:
gs_cols = re.findall('GS_[A-Z]+', ' '.join(table.columns))

In [21]:
gs = pd.concat([table.iloc[:, :3], table[gs_cols], table['AMT_IND']], axis=1) \
       .groupby([pd.Grouper(freq='MS'), 'HDONG_NM'])[gs_cols+['AMT_IND']].mean() \
       .reset_index().set_index('STD_YMD')

In [22]:
gs.loc[:, 'GS_MEAL':'GS_PREG'] = gs.loc[:, 'GS_MEAL':'GS_PREG'].apply(lambda x: x*gs['AMT_IND'])

In [23]:
series_list = eda.compared_to_previous(gs, gs_cols)

작년 대비 올해의 편의점 각 품목당 변화율이 유의미한지 일표본 t검정으로 테스트

유의수준 0.01에서 t검정을 진행하였으며, 해당 유의수준보다 작은 값들만 signif에 저장

In [24]:
signif = []
for i in range(len(series_list)):
    p_value = stats.ttest_1samp(series_list[i], 0)[1]
    if p_value <= 0.01:
        signif.append(series_list[i].name)

In [25]:
signif_features = pd.concat(series_list, axis=1)[signif]

In [26]:
hdong_series = pd.Series(table['HDONG_NM'].unique(), name='HDONG_NM')

In [27]:
tmp = pd.concat([hdong_series, signif_features], axis=1)

In [28]:
tmp

,HDONG_NM,GS_MEAL_FEB,GS_MEAL_MAR,GS_MEAL_APR,GS_MEAL_MAY,GS_SNACK_MAR,GS_SNACK_APR,GS_DRINK_FEB,GS_DRINK_MAR,GS_DRINK_APR,...,GS_HEALTH_APR,GS_HEALTH_MAY,GS_HOBBY_FEB,GS_HOBBY_MAR,GS_HOBBY_APR,GS_HOBBY_MAY,GS_SOCIAL_FEB,GS_SOCIAL_MAR,GS_SOCIAL_APR,GS_SOCIAL_MAY
0,고산1동,-0.242773,-0.293172,-0.322137,-0.227082,-0.122815,-0.220081,-0.197207,-0.270233,-0.259623,...,-0.177794,0.131224,-0.438154,-0.364824,0.105232,-0.372925,-0.471470,-0.517012,-0.365696,0.028787
1,고산2동,-0.127204,-0.090927,0.022664,0.002681,0.216333,0.174442,-0.059997,0.033491,0.024409,...,-0.056803,0.211919,-0.715958,0.279319,-0.445428,-0.166979,-0.286340,-0.395283,-0.160361,0.022968
2,고산3동,-0.285882,-0.336911,-0.281355,-0.279646,-0.129016,-0.138689,-0.260403,-0.235157,-0.163808,...,-0.167301,0.094337,-0.289761,0.425576,0.078722,-0.513768,-0.385849,-0.497378,-0.291894,-0.374321
3,두산동,-0.049328,-0.152916,-0.142494,-0.113312,-0.129117,-0.129966,-0.084994,-0.213950,-0.198487,...,-0.015275,0.236900,-0.215470,-0.255345,-0.418494,-0.246504,-0.228990,-0.414225,-0.307877,-0.155372
4,만촌1동,0.017247,-0.112096,-0.091803,-0.012847,-0.001483,-0.028962,0.017918,-0.095436,-0.124882,...,-0.145157,0.129936,-0.307326,-0.289548,-0.310355,-0.226817,-0.159375,-0.426811,-0.467369,-0.104110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,중림동,-0.104348,-0.117075,-0.070149,-0.047741,0.060386,0.041341,-0.072628,-0.021808,-0.035377,...,-0.069218,0.118365,-0.344012,-0.418720,-0.344419,-0.282621,-0.382675,-0.342624,-0.347170,-0.139545
65,청구동,-0.221918,-0.310022,-0.250853,-0.111496,0.055431,0.009076,-0.036571,-0.054805,-0.087063,...,0.030768,0.122514,-0.684423,-0.337905,-0.223192,-0.650486,-0.230067,-0.323693,-0.495378,-0.273270
66,필동,-0.061139,-0.141069,-0.158065,-0.104612,-0.157698,-0.087358,-0.084928,-0.270376,-0.193248,...,-0.142893,-0.006245,-0.081470,-0.281277,-0.158019,-0.095301,-0.313470,-0.370703,-0.188720,-0.063299
67,황학동,0.125184,-0.031916,-0.088033,-0.019638,0.130929,-0.007457,0.067974,-0.001141,-0.049771,...,-0.023457,0.212166,0.121669,0.414500,-0.267134,-0.297000,-0.214827,-0.165910,-0.181732,0.091826


**CJ**

In [29]:
cj_cols = re.findall('CJ_[A-Z]+', ' '.join(table.columns))

In [30]:
cj = pd.concat([table.iloc[:, :3], table[cj_cols], table['INV_CNT']], axis=1) \
       .groupby([pd.Grouper(freq='MS'), 'HDONG_NM'])[cj_cols+['INV_CNT']].mean() \
       .reset_index().set_index('STD_YMD')

In [31]:
scaler = 1 / cj.loc[:, 'CJ_FURN':'CJ_BEAUTY'].sum(axis=1)

In [32]:
cj.iloc[:, 1:-1] = cj.iloc[:, 1:-1].apply(lambda x: x*scaler)

In [33]:
cj.iloc[:, 1:-1] = cj.iloc[:, 1:-1].apply(lambda x: x*cj['INV_CNT'])

In [34]:
series_list = eda.compared_to_previous(cj, cj_cols)

In [35]:
signif = []
for i in range(len(series_list)):
    p_value = stats.ttest_1samp(series_list[i], 0)[1]
    if p_value <= 0.01:
        signif.append(series_list[i].name)

In [36]:
signif_features = pd.concat(series_list, axis=1)[signif]

In [37]:
tmp = pd.concat([tmp, signif_features], axis=1)

In [38]:
tmp

,HDONG_NM,GS_MEAL_FEB,GS_MEAL_MAR,GS_MEAL_APR,GS_MEAL_MAY,GS_SNACK_MAR,GS_SNACK_APR,GS_DRINK_FEB,GS_DRINK_MAR,GS_DRINK_APR,...,CJ_CLOTH_FEB,CJ_CLOTH_MAR,CJ_CLOTH_MAY,CJ_ACC_FEB,CJ_ACC_MAR,CJ_ACC_MAY,CJ_BEAUTY_FEB,CJ_BEAUTY_MAR,CJ_BEAUTY_APR,CJ_BEAUTY_MAY
0,고산1동,-0.242773,-0.293172,-0.322137,-0.227082,-0.122815,-0.220081,-0.197207,-0.270233,-0.259623,...,0.164089,0.057803,0.230380,0.250757,0.102200,0.216438,0.462547,0.329869,0.099614,0.194213
1,고산2동,-0.127204,-0.090927,0.022664,0.002681,0.216333,0.174442,-0.059997,0.033491,0.024409,...,0.306254,0.196791,0.289680,0.434536,0.352817,0.425727,0.578649,0.491270,0.324844,0.291870
2,고산3동,-0.285882,-0.336911,-0.281355,-0.279646,-0.129016,-0.138689,-0.260403,-0.235157,-0.163808,...,0.074906,0.113369,0.225900,-0.050209,0.254264,0.219999,0.204557,0.438543,0.057045,0.095687
3,두산동,-0.049328,-0.152916,-0.142494,-0.113312,-0.129117,-0.129966,-0.084994,-0.213950,-0.198487,...,0.139929,0.124901,0.204495,0.172104,0.103355,0.166930,0.152855,0.155085,0.044071,0.094264
4,만촌1동,0.017247,-0.112096,-0.091803,-0.012847,-0.001483,-0.028962,0.017918,-0.095436,-0.124882,...,0.111427,-0.029126,0.038216,0.281284,0.013427,0.085541,0.151662,0.118610,-0.043256,-0.027796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,중림동,-0.104348,-0.117075,-0.070149,-0.047741,0.060386,0.041341,-0.072628,-0.021808,-0.035377,...,0.070544,0.090694,0.147873,0.181701,0.194789,0.115658,0.232619,0.344990,0.049580,0.153680
65,청구동,-0.221918,-0.310022,-0.250853,-0.111496,0.055431,0.009076,-0.036571,-0.054805,-0.087063,...,0.311077,0.173635,0.271368,0.252459,0.422120,0.284285,0.502018,0.440040,0.181053,0.150910
66,필동,-0.061139,-0.141069,-0.158065,-0.104612,-0.157698,-0.087358,-0.084928,-0.270376,-0.193248,...,0.077595,0.069254,0.291442,0.384985,0.341775,0.352729,0.307905,0.246781,0.017896,0.098749
67,황학동,0.125184,-0.031916,-0.088033,-0.019638,0.130929,-0.007457,0.067974,-0.001141,-0.049771,...,0.160000,0.101519,0.123751,0.661144,0.250404,0.470631,0.416364,0.309980,0.279312,0.260725


In [ ]:
#table.to_csv('../Data/table.csv', index=False)

=============================================

age_cols = re.findall('AG_[0-9]{2}_[MF]', ' '.join(table.columns))<br>
table[age_cols].sum(axis-=1).sum()

In [58]:
age_cols = re.findall('AG_[0-9]{2}_[MF]', ' '.join(table.columns))

In [82]:
age = pd.concat([table.iloc[:,:2], table[age_cols], table['POP_CNT']], axis=1)\
        .groupby([pd.Grouper(freq='MS'), 'HDONG_NM'])[age_cols+['POP_CNT']].mean() \
        .reset_index().set_index('STD_YMD')

In [85]:
scaler = 1 / age.loc[:,'AG_20_F':'AG_60_M'].sum(axis=1)

In [86]:
age.iloc[:, 1:-1] = age.iloc[:, 1:-1].apply(lambda x: x*scaler)

In [87]:
age.iloc[:, 1:-1] = age.iloc[:, 1:-1].apply(lambda x: x*age['POP_CNT'])

In [116]:
age_series_list = eda.compared_to_previous(age, age_cols)

In [120]:
signif = []
for i in range(len(age_series_list)):
    p_value = stats.ttest_1samp(age_series_list[i], 0)[1]
    if p_value <= 0.01:
        signif.append(age_series_list[i].name)

In [121]:
signif_features = pd.concat(age_series_list, axis=1)[signif]

In [122]:
tmp = pd.concat([tmp, signif_features], axis=1)

In [123]:
tmp

,HDONG_NM,GS_MEAL_FEB,GS_MEAL_MAR,GS_MEAL_APR,GS_MEAL_MAY,GS_SNACK_MAR,GS_SNACK_APR,GS_DRINK_FEB,GS_DRINK_MAR,GS_DRINK_APR,...,AG_40_M_APR,AG_40_M_MAY,AG_50_M_FEB,AG_50_M_MAR,AG_50_M_APR,AG_50_M_MAY,AG_60_M_FEB,AG_60_M_MAR,AG_60_M_APR,AG_60_M_MAY
0,고산1동,-0.242773,-0.293172,-0.322137,-0.227082,-0.122815,-0.220081,-0.197207,-0.270233,-0.259623,...,0.012613,-0.081806,-0.052698,-0.138416,0.058754,-0.037820,-0.017185,-0.167002,0.086321,-0.004738
1,고산2동,-0.127204,-0.090927,0.022664,0.002681,0.216333,0.174442,-0.059997,0.033491,0.024409,...,-0.141025,-0.086729,-0.120803,-0.216469,-0.086882,-0.010821,-0.058043,-0.183833,-0.044063,0.053788
2,고산3동,-0.285882,-0.336911,-0.281355,-0.279646,-0.129016,-0.138689,-0.260403,-0.235157,-0.163808,...,-0.111939,-0.049366,-0.112961,-0.235750,-0.071919,-0.007798,-0.053841,-0.224763,0.009923,0.054551
3,두산동,-0.049328,-0.152916,-0.142494,-0.113312,-0.129117,-0.129966,-0.084994,-0.213950,-0.198487,...,-0.051319,-0.029442,-0.159741,-0.221455,-0.071182,-0.036757,-0.140168,-0.239130,-0.032104,0.018204
4,만촌1동,0.017247,-0.112096,-0.091803,-0.012847,-0.001483,-0.028962,0.017918,-0.095436,-0.124882,...,-0.134711,-0.094619,-0.187546,-0.253194,-0.092836,-0.056656,-0.201908,-0.261758,-0.034817,-0.023937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,중림동,-0.104348,-0.117075,-0.070149,-0.047741,0.060386,0.041341,-0.072628,-0.021808,-0.035377,...,-0.137527,-0.086968,-0.230990,-0.299351,-0.133922,-0.068988,-0.261605,-0.366656,-0.123729,-0.024007
65,청구동,-0.221918,-0.310022,-0.250853,-0.111496,0.055431,0.009076,-0.036571,-0.054805,-0.087063,...,-0.217846,-0.117380,-0.180450,-0.294518,-0.182692,-0.103034,-0.127370,-0.301784,-0.174007,-0.091803
66,필동,-0.061139,-0.141069,-0.158065,-0.104612,-0.157698,-0.087358,-0.084928,-0.270376,-0.193248,...,-0.199723,-0.115905,-0.079924,-0.380322,-0.192730,-0.086684,-0.086422,-0.420120,-0.201890,-0.091975
67,황학동,0.125184,-0.031916,-0.088033,-0.019638,0.130929,-0.007457,0.067974,-0.001141,-0.049771,...,-0.124025,-0.165088,-0.180259,-0.280503,-0.109998,-0.157230,-0.228943,-0.349151,-0.123302,-0.179200


시간대별

In [98]:
time_cols = re.findall('TMST_[0-9]{2}', ' '.join(table.columns))

In [100]:
time = pd.concat([table.iloc[:,:2], table[time_cols], table['POP_CNT']], axis=1)\
        .groupby([pd.Grouper(freq='MS'), 'HDONG_NM'])[time_cols+['POP_CNT']].mean() \
        .reset_index().set_index('STD_YMD')

In [102]:
scaler = 1 / time.loc[:,'TMST_01':'TMST_23'].sum(axis=1)

In [103]:
time.iloc[:, 1:-1] = time.iloc[:, 1:-1].apply(lambda x: x*scaler)

In [104]:
time.iloc[:, 1:-1] = time.iloc[:, 1:-1].apply(lambda x: x*age['POP_CNT'])

In [110]:
time_series_list = eda.compared_to_previous(time, time_cols)

In [119]:
signif = []
for i in range(len(time_series_list)):
    p_value = stats.ttest_1samp(time_series_list[i], 0)[1]
    if p_value <= 0.01:
        signif.append(time_series_list[i].name)

In [112]:
signif_features = pd.concat(time_series_list, axis=1)[signif]

In [114]:
tmp = pd.concat([tmp, signif_features], axis=1)

In [115]:
tmp

,HDONG_NM,GS_MEAL_FEB,GS_MEAL_MAR,GS_MEAL_APR,GS_MEAL_MAY,GS_SNACK_MAR,GS_SNACK_APR,GS_DRINK_FEB,GS_DRINK_MAR,GS_DRINK_APR,...,TMST_21_MAR,TMST_21_APR,TMST_21_MAY,TMST_22_FEB,TMST_22_MAR,TMST_22_APR,TMST_22_MAY,TMST_23_FEB,TMST_23_APR,TMST_23_MAY
0,고산1동,-0.242773,-0.293172,-0.322137,-0.227082,-0.122815,-0.220081,-0.197207,-0.270233,-0.259623,...,-0.209930,-0.118227,-0.229922,0.170579,-0.121443,-0.130087,-0.248332,0.490815,0.003633,-0.185048
1,고산2동,-0.127204,-0.090927,0.022664,0.002681,0.216333,0.174442,-0.059997,0.033491,0.024409,...,-0.339901,-0.344921,-0.264906,-0.041151,-0.256346,-0.350518,-0.261254,0.209850,-0.336626,-0.228401
2,고산3동,-0.285882,-0.336911,-0.281355,-0.279646,-0.129016,-0.138689,-0.260403,-0.235157,-0.163808,...,-0.203678,-0.153929,-0.089930,0.079190,-0.126378,-0.176078,-0.088861,0.398202,-0.003964,0.154200
3,두산동,-0.049328,-0.152916,-0.142494,-0.113312,-0.129117,-0.129966,-0.084994,-0.213950,-0.198487,...,-0.306030,-0.152486,-0.112827,-0.159354,-0.319562,-0.213580,-0.160604,-0.017662,-0.183402,-0.156097
4,만촌1동,0.017247,-0.112096,-0.091803,-0.012847,-0.001483,-0.028962,0.017918,-0.095436,-0.124882,...,-0.374841,-0.266358,-0.210879,-0.180349,-0.324674,-0.294658,-0.229584,-0.117101,-0.253994,-0.195829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,중림동,-0.104348,-0.117075,-0.070149,-0.047741,0.060386,0.041341,-0.072628,-0.021808,-0.035377,...,-0.361788,-0.216852,-0.150864,-0.122612,-0.309680,-0.252731,-0.190144,-0.038602,-0.287588,-0.236074
65,청구동,-0.221918,-0.310022,-0.250853,-0.111496,0.055431,0.009076,-0.036571,-0.054805,-0.087063,...,-0.541620,-0.390670,-0.241797,-0.199688,-0.497190,-0.457593,-0.291204,-0.198890,-0.463875,-0.341557
66,필동,-0.061139,-0.141069,-0.158065,-0.104612,-0.157698,-0.087358,-0.084928,-0.270376,-0.193248,...,-0.475725,-0.361393,-0.232623,0.087952,-0.394598,-0.398916,-0.236433,0.319157,-0.359280,-0.197920
67,황학동,0.125184,-0.031916,-0.088033,-0.019638,0.130929,-0.007457,0.067974,-0.001141,-0.049771,...,-0.319472,-0.161743,-0.182799,-0.140046,-0.325550,-0.294617,-0.270552,0.065591,-0.253075,-0.231653
